# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
cities_df = pd.read_csv('../output_data/cities.csv')
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Rikitea,-23.12,-134.97,70.23,77,0,18.32,PF,1595792995
1,Kysyl-Syr,63.90,122.76,51.80,90,25,1.21,RU,1595792785
2,Astoria,46.19,-123.83,80.60,44,1,12.75,US,1595792786
3,Bereda,43.27,-7.54,71.01,78,0,7.00,ES,1595792742
4,Ganzhou,28.82,115.43,77.83,90,100,2.66,CN,1595792786


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [6]:
gmaps.configure(api_key=g_key)

In [7]:
# Customize the size of the figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [8]:
# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Weight
humidity = cities_df['Humidity']

# Plot Heatmap
fig = gmaps.figure(center=(0.0,0.0),zoom_level=2)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
cleaned_df = cities_df.dropna(how='any')

In [10]:
ideal_temp = cleaned_df.loc[(cleaned_df['Max Temp']>65)&(cleaned_df['Max Temp']<75)]

In [11]:
hotel_df = ideal_temp.loc[(ideal_temp['Humidity']<50)&(ideal_temp['Cloudiness']<60)&(ideal_temp['Wind Speed']<5)]
hotel_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
76,Kampene,-3.60,26.67,72.81,37,0,1.59,CD,1595792813
323,Guaranésia,-21.30,-46.80,73.85,47,0,4.52,BR,1595792902
456,Abhar,36.15,49.22,72.23,34,0,0.76,IR,1595792948


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
hotel_df['Hotel Name'] = ''

C:\Users\nabis\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "type": "hotel",
    "radius": 5000,
    "key": g_key}
for index, row in hotel_df.iterrows():
    
    # get the lat and long for each city and form location parameter
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    
    # assemble query url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except:
        print('Sorry could not find a hotel within parameters entered.')

# hotel_df.drop(columns=['Max Temp','Humidity', 'Cloudiness', 'Wind Speed', 'Date'], inplace=True)
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Hotel Name']]
hotel_df.head()

C:\Users\nabis\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,City,Country,Lat,Lng,Hotel Name
76,Kampene,CD,-3.60,26.67,Kampene
323,Guaranésia,BR,-21.30,-46.80,Guaxupé
456,Abhar,IR,36.15,49.22,Abhar


In [14]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [15]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)

# Add layer
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))